Устанавливаем библиотеку для работы с gpu с помощью данной команды:

pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu117

In [1]:
import torch
# проверяем увидила ли наша библиотека gpu устройство
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
if device == torch.device("cuda"):
    device = 0

cuda


Учитывая ограничение по времени и мощности будем использовать библиотеку YOLO. Это открытая библиотека с уже предообученной моделью, что упростит нам задачу и время, мы будем просто переобучать её на наши данные, а не обучать с нуля (Что заняло бы достаточно времени). Я выбрал модель yoloV8 так как она более современная и подерживает работу не только через терминал, по сравнению с yolov5.

Устанавливаем сопуствующие пакеты к библиотеке.
pip install ultralytics - сама библиотека содержащая все модели yolo

pip install supervision - полезный пакет для взаимодействия с камерой

Устанавливаем Библиотеку scikit-learn, в ней есть полезные методы для машинного обучения, а также функции упрощающие работу.
 
pip install scikit-learn

In [15]:
import os
# Получаем текущую дирректорию
os.getcwd()
# считываем наши файлы в списки изображений и разметки 
images = [os.path.join('../all/images/', x) for x in os.listdir('../all/images')if x.endswith('.jpg')]
labels = [os.path.join('../all/images/', x) for x in os.listdir('../all/images')if x.endswith('.txt')]
# сортируем их
images.sort()
labels.sort()

In [16]:
from sklearn.model_selection import train_test_split
# производим разделение всех наших данных на Тренировочные и валидационные
train_images, val_images, train_annotations, val_annotations = train_test_split(images, labels, test_size=0.15, random_state=42)
# валидационные данные делим на тестовые и валидационные, чтобы одни использовать для тестирования пока обучается, а вторые для тестирования после обучения
val_images, test_images, val_annotations, test_annotations = train_test_split(val_images, val_annotations, test_size=0.5, random_state=42)

In [17]:
# Создаем папки куда будем переносить наши разбитые на 3 набора данные фотографий
parent = 'new'
dirs = 'images/train', 'images/val', 'images/test', 'labels/train', 'labels/val', 'labels/test'
for dir in dirs:
    path = os.path.join(parent, dir)
    # проверяем существование папки чтобы не создавать если уже есть
    if not os.path.exists(path):
        # создаем папку
        os.makedirs(path)

In [18]:
import shutil
#Утилитная функция для переноса файлов 
def move_files_to_folder(list_of_files, destination_folder):
    for f in list_of_files:
        try:
            shutil.copy(f, destination_folder)
        except:
            print(f)
            assert False

# Переносим файлы по из папкам учитывая разделение
move_files_to_folder(train_images, 'new/images/train')
move_files_to_folder(val_images, 'new/images/val/')
move_files_to_folder(test_images, 'new/images/test/')
move_files_to_folder(train_annotations, 'new/labels/train/')
move_files_to_folder(val_annotations, 'new/labels/val/')
move_files_to_folder(test_annotations, 'new/labels/test/')

Теперь создаем файл data.yaml.
В этом файле хранится информация путей к нашим данным для обучения и тренировки, а также классы которые у нас есть.

In [19]:
#  импортируем YOLO класс из библиотеки ultralytics
from ultralytics import YOLO

# Создаем новую YOLO модель с нуля
model = YOLO('yolov8n.yaml')
# загружаем претренированную YOLO модель
model = YOLO('yolov8n.pt')


                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.Conv                  [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.Conv                  [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.C2f                   [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.Conv                  [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.C2f                   [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.Conv                  [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.C2f                   [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics.nn.modules.Conv                  [128

переносим по данному пути
 ['C:\\Users\\igorv\\OneDrive\\Documents\\GitHub\\datasets\\new\\images\\val']

In [21]:
# Трнируем модель на основе 'data.yaml' файла нашего датасета на 10 эпох
results = model.train(data='data_new.yaml', epochs=10,  device=device, name='custom')

New https://pypi.org/project/ultralytics/8.0.83 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.70  Python-3.10.9 torch-2.0.0+cu117 CUDA:0 (NVIDIA GeForce RTX 2070 Super, 8192MiB)
yolo\engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=data_new.yaml, epochs=10, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=custom, exist_ok=False, pretrained=False, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, image_weights=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_thickness=3, visualize=False, augment=False, agnostic_nms=False, classes=None, ret

In [22]:
# Трнируем модель на основе 'data.yaml' файла нашего датасета на 100 эпох
results = model.train(data='data_new.yaml', epochs=150,  device=device, name='custom')

New https://pypi.org/project/ultralytics/8.0.83 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.70  Python-3.10.9 torch-2.0.0+cu117 CUDA:0 (NVIDIA GeForce RTX 2070 Super, 8192MiB)
yolo\engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=data_new.yaml, epochs=150, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=custom, exist_ok=False, pretrained=False, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, image_weights=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_thickness=3, visualize=False, augment=False, agnostic_nms=False, classes=None, re

In [23]:
# Вычисляем точность модели на проверочном наборе данных
results = model.val()

Ultralytics YOLOv8.0.70  Python-3.10.9 torch-2.0.0+cu117 CUDA:0 (NVIDIA GeForce RTX 2070 Super, 8192MiB)
Model summary (fused): 168 layers, 3006818 parameters, 0 gradients, 8.1 GFLOPs
val: Scanning C:\Users\igorv\OneDrive\Documents\GitHub\datasets\new\labels\val.cache... 253 images, 90 backgrounds, 6 corrupt: 100%|██████████| 253/253 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 16/16 [00:51<00:00,  3.25s/it]
                   all        253        411      0.599      0.364      0.393      0.237
                   D00        253        203      0.598      0.236      0.309       0.15
                   D10        253         88      0.625       0.33      0.375      0.177
                   D20        253         71      0.605      0.479      0.539      0.266
                   D40        253         44      0.606      0.175      0.212     0.0998
                Repair        253          5      0.562        0

В качестве API создаем файл camera.py, содержащий функцию main(). И запускающий детекцию дефектов дорожного покрытия. В режиме реального времени. 

python3 -m pip install pytest-cov